# what variables will we use for image-processing

takeaway/resume: 
In the questionnaire,
* some answers can be answered per uploading a picture instead of answering per text:
1. 'Body Type'
2. ,'Sex'
3. ,'Diet'
4. ,'Heating Energy Source'
5. ,'Transport'
6. ,'Vehicle Type'
7. ,'Waste Bag Size'
8. ,'Waste Bag Weekly Count'
9. ,'How Many New Clothes Monthly'
10. 'Energy Efficiency'
11. ,'Recycling'
12. ,'Cooking_With'
* the answer can NOT be answered per uploading a picture:
13. how often Shower
14. Social-Activity
15. how often Air-Travel
16. Grocery_Bill
17. Vehicle_Distance
18. TV/day
19. Internet/day

But if i only use Variables 1-12. the R2-score is really bad (56%), so we will have to do a mix of pictures and text-answering.

### Importing the dataset and libraries


we have the data from: https://www.kaggle.com/datasets/dumanmesut/individual-carbon-footprint-calculation/data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

# Change display settings to show all columns
pd.set_option('display.max_columns', None)

df=pd.read_csv('Carbon_Emission.csv',sep=";")
df.head()

,Body Type,Sex,Diet,How Often Shower,Heating Energy Source,Transport,Vehicle Type,Social Activity,Monthly Grocery Bill,Frequency of Traveling by Air,Vehicle Monthly Distance Km,Waste Bag Size,Waste Bag Weekly Count,How Long TV PC Daily Hour,How Many New Clothes Monthly,How Long Internet Daily Hour,Energy efficiency,Recycling,Cooking_With,CarbonEmission
0,overweight,female,pescatarian,daily,coal,public,NaN,often,230,frequently,210,large,4,7,26,1,No,['Metal'],"['Stove', 'Oven']",2238
1,obese,female,vegetarian,less frequently,natural gas,walk/bicycle,NaN,often,114,rarely,9,extra large,3,9,38,5,No,['Metal'],"['Stove', 'Microwave']",1892
2,overweight,male,omnivore,more frequently,wood,private,petrol,never,138,never,2472,small,1,14,47,6,Sometimes,['Metal'],"['Oven', 'Microwave']",2595
3,overweight,male,omnivore,twice a day,wood,walk/bicycle,NaN,sometimes,157,rarely,74,medium,3,20,5,7,Sometimes,"['Paper', 'Plastic', 'Glass', 'Metal']","['Microwave', 'Grill', 'Airfryer']",1074
4,obese,female,vegetarian,daily,coal,private,diesel,often,266,very frequently,8457,large,1,3,5,6,Yes,['Paper'],['Oven'],4743


# variables "Transport" and "Vehicle Type" (they have combined information)


In [2]:

##create new column: 
df["Transport Vehicle Type"]=df["Vehicle Type"] #definiere neue Spalte
df.loc[df["Transport Vehicle Type"].isna(), "Transport Vehicle Type"] = df["Transport"] # Werte aus 'Transport' übernehmen, wenn 'Vehicle Type' NaN ist


##veranschaulichen der neuen Spalten und ihrer Werte
df[["Transport","Vehicle Type","Transport Vehicle Type"]]


,Transport,Vehicle Type,Transport Vehicle Type
0,public,NaN,public
1,walk/bicycle,NaN,walk/bicycle
2,private,petrol,petrol
3,walk/bicycle,NaN,walk/bicycle
4,private,diesel,diesel
...,...,...,...
9995,private,hybrid,hybrid
9996,private,lpg,lpg
9997,walk/bicycle,NaN,walk/bicycle
9998,private,petrol,petrol


# variables "Recycling" and "Cooking_With" (these variables allow multiple answers)

In [4]:

##create new column: dummy-variables for Recycling
unique_values_Recycling= set([item for sublist in df['Recycling'].unique() for item in eval(sublist)]) # Liste aller einzigartigen Recycling-Möglichkeiten
# Für jede einzigartige Recycling-Möglichkeit eine neue Spalte hinzufügen und mit 0 oder 1 füllen
for item in unique_values_Recycling:
    df['Recycling '+str(item)] = df['Recycling'].apply(lambda x: 1 if item in x else 0)

##create new column: dummy-variables for Cooking-with
unique_values_cooking_With= set([item for sublist in df['Cooking_With'].unique() for item in eval(sublist)]) # Liste aller einzigartigen Cooking_With-Möglichkeiten
# Für jede einzigartige Cooking_With-Möglichkeit eine neue Spalte hinzufügen und mit 0 oder 1 füllen
for item in unique_values_cooking_With:
    df['Cooking With '+str(item)] = df['Cooking_With'].apply(lambda x: 1 if item in x else 0)


#Person 282 cooks with "nothing" deshalb can bei one-hot-encoding nicht eine Spalte gelöscht werden, person 283 who doesn't recycle deshalb can bei one-hot-encoding nicht eine Spalte gelöscht werden
#Darstellung 
df.iloc[282:284, -13:] 

,Recycling,Cooking_With,CarbonEmission,Transport Vehicle Type,Recycling Plastic,Recycling Paper,Recycling Glass,Recycling Metal,Cooking With Oven,Cooking With Grill,Cooking With Microwave,Cooking With Stove,Cooking With Airfryer
282,"['Paper', 'Plastic', 'Metal']",[],1484,public,1,1,0,1,0,0,0,0,0
283,[],"['Stove', 'Grill', 'Airfryer']",2955,public,0,0,0,0,0,1,0,1,1


# sorting the variable into groups 

In [5]:
for_image_processing=['Body Type'
,'Sex'
,'Diet'
,'Heating Energy Source'
,'Transport'
,'Vehicle Type'
,'Waste Bag Size'
,'Waste Bag Weekly Count'
,'How Many New Clothes Monthly'
,'Energy efficiency'
,'Recycling'
,'Cooking_With']


variables_quantitative=df[for_image_processing].select_dtypes(include=[np.number]).columns.tolist()

variables_for_one_hot_encoded=list(set(df[for_image_processing].select_dtypes(include=['object','category']).columns.tolist()) - {'Transport','Vehicle Type','Recycling','Cooking_With'})


##add "Transport Vehicle Type" to "one-hot-encoding"-list
if "Transport Vehicle Type" not in variables_for_one_hot_encoded: ##wenn es noch nicht hinzugefügt wurde bzw is one element in the list already?
    variables_for_one_hot_encoded.append("Transport Vehicle Type") 


##add dummy-varialbes for 'Recycling' and 'Cooking With' to "variables_quantitative"-list for Regression
columns_recycling=['Recycling '+str(item) for item in unique_values_Recycling] #liste mit Spalten-Namen
if columns_recycling[0] not in variables_quantitative: ##wenn es noch nicht hinzugefügt wurde bzw is one element in the list already?
    variables_quantitative += columns_recycling
columns_cooking_with=['Cooking With '+str(item) for item in unique_values_cooking_With] #liste mit Spalten-Namen
if columns_cooking_with[0] not in variables_quantitative: ##wenn es noch nicht hinzugefügt wurde bzw is one element in the list already?
    variables_quantitative += columns_cooking_with


### One-Hot-Encoding for categorical variables

In [6]:
#one-hot-encoding

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

X = df[variables_quantitative + variables_for_one_hot_encoded]   #Ziel: every column except "CarbonEmission"

# Create the ColumnTransformer using the list "variables_for_one_hot_encoded"
cf = ColumnTransformer(
    [(col, OneHotEncoder(drop="first"), [col]) for col in variables_for_one_hot_encoded],  
    remainder="passthrough")   


cf.fit(X)
X_transformed = cf.transform(X)

#  Split the data into training and testing sets

In [7]:

y = df["CarbonEmission"]


    ##Splitting the dataset into train and test set
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X_transformed,y, train_size = 0.75)


# model for image-processing

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression, ElasticNetCV, LassoCV, LassoLarsIC, LassoLarsCV, Lars, LarsCV, RidgeCV, BayesianRidge, Ridge, SGDRegressor, HuberRegressor, TweedieRegressor, ElasticNet, Lasso
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.svm import SVR, NuSVR, LinearSVR
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor, HistGradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import PoissonRegressor, GammaRegressor, RANSACRegressor, SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
#from sklearn.linear_model import LinearSVR
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import TweedieRegressor
#from sklearn.linear_model import GeneralizedLinearRegressor 
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import OrthogonalMatchingPursuitCV, PassiveAggressiveRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.kernel_ridge import KernelRidge

# Initialize all the regressors
models = {
    "Linear Regression": LinearRegression(),
    "ElasticNetCV": ElasticNetCV(),
    "LassoCV": LassoCV(),
    "LassoLarsIC": LassoLarsIC(),
    "LassoLarsCV": LassoLarsCV(),
    "Lars": Lars(),
    "LarsCV": LarsCV(),
    "RidgeCV": RidgeCV(),
    "BayesianRidge": BayesianRidge(),
    "Ridge": Ridge(),
    "SGDRegressor": SGDRegressor(),
    "HuberRegressor": HuberRegressor(),
    "TweedieRegressor": TweedieRegressor(),
    "ElasticNet": ElasticNet(),
    "Lasso": Lasso(),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    "ExtraTreeRegressor": ExtraTreeRegressor(),
    "SVR": SVR(),
    "NuSVR": NuSVR(),
    #"LinearSVR": LinearSVR(),
    "RandomForestRegressor": RandomForestRegressor(),
    "BaggingRegressor": BaggingRegressor(),
    "GradientBoostingRegressor": GradientBoostingRegressor(),
    "ExtraTreesRegressor": ExtraTreesRegressor(),
    "AdaBoostRegressor": AdaBoostRegressor(),
    "HistGradientBoostingRegressor": HistGradientBoostingRegressor(),
    "PoissonRegressor": PoissonRegressor(),
    "LGBMRegressor": LGBMRegressor(),
    "KNeighborsRegressor": KNeighborsRegressor(),
    "MLPRegressor": MLPRegressor(),
    "GammaRegressor": GammaRegressor(),
    "RANSACRegressor": RANSACRegressor(),
    "OrthogonalMatchingPursuitCV": OrthogonalMatchingPursuitCV(),
    "PassiveAggressiveRegressor": PassiveAggressiveRegressor(),
    "GaussianProcessRegressor": GaussianProcessRegressor(),
    #"OrthogonalMatchingPursuit": OrthogonalMatchingPursuit(),
    "ExtraTreeRegressor": ExtraTreeRegressor(),
    "DummyRegressor": DummyRegressor(),
    #"LassoLars": LassoLars(),
    "KernelRidge": KernelRidge()
}

# Fit all the models and evaluate
results = []
for name, model in models.items():
    try:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        r2 = r2_score(y_test, y_pred)  #r2 = round(r2_score(y_test, y_pred),5)
        mae = mean_absolute_error(y_test, y_pred)
        train_score = model.score(X_train, y_train)
        results.append((name, r2, mae, train_score))
    except Exception as e:
        results.append((name, "Error", str(e), "Error"))

# Create a dataframe to display results
results_df = pd.DataFrame(results, columns=['Model', 'R2 Score', 'mean absolute error', 'Train Score'])
results_df.sort_values(by='R2 Score', ascending=False, inplace=True) #sort by R2-Score
results_df

/opt/homebrew/Caskroom/miniforge/base/envs/elisabeth_python-umgebung_nr1/lib/python3.11/site-packages/sklearn/linear_model/_least_angle.py:755: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 30 iterations, alpha=6.293e+00, previous alpha=6.218e+00, with an active set of 29 regressors.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/elisabeth_python-umgebung_nr1/lib/python3.11/site-packages/sklearn/linear_model/_least_angle.py:725: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 21 iterations, i.e. alpha=1.385e+01, with an active set of 21 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/elisabeth_python-umgebung_nr1/lib/python3.11/site-packages/sklearn/linear_model/_least_angle.py:725: ConvergenceWarning: Regressors in active s

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000625 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 119
[LightGBM] [Info] Number of data points in the train set: 7500, number of used features: 32
[LightGBM] [Info] Start training from score 2272.083733


/opt/homebrew/Caskroom/miniforge/base/envs/elisabeth_python-umgebung_nr1/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


,Model,R2 Score,mean absolute error,Train Score
8,BayesianRidge,0.557090,540.402039,0.569117
7,RidgeCV,0.557040,540.379347,0.569134
9,Ridge,0.557040,540.379347,0.569134
0,Linear Regression,0.556985,540.363856,0.569139
3,LassoLarsIC,0.556984,540.364474,0.569139
4,LassoLarsCV,0.556984,540.364474,0.569139
5,Lars,0.556984,540.364474,0.569139
6,LarsCV,0.556984,540.364474,0.569139
14,Lasso,0.556589,540.746594,0.568811
25,PoissonRegressor,0.555338,539.610343,0.573409
